This notebook reads a DVH for a CTV and returns the TCP.

In [ ]:
#Import relevant libraries
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad

In [ ]:
#Read the Cummulative DVH and convert it into a differential DVH

#Define lists for dose values and their corresponding volume
doses = []
vol = []

#Read the CSV file
with open('/content/CTV70_LET.csv', newline='') as csvfile:
  reader = csv.DictReader(csvfile)

  #Add relevant data to the lists
  for row in reader:
    doses.append(float(row['Dose']))
    vol.append(float(row['Percentage']))

#Etablish lists for the differential DVH parameters
diffVol = []
diffDos = []

#Determine the differential parameters
for i in range(1,len(vol)):
  diffDos.append(doses[i-1]+(doses[i]-doses[i-1])/2)
  diffVol.append(round(((vol[i-1]-vol[i])/100) * 104.330/(0.3*0.1953125*0.1953125)))

In [ ]:
def SF(dose,alph,ab):
  '''For a given dose per fraction, alpha value and alpha/beta ratio, this 
  funtion returns the survivng fraction of a 35 fraction treatment'''
  
  return np.exp(-alph*dose*35*(1+dose/ab))

In [ ]:
def TCP(dos,vol,size,alph,ab):
  '''This function determines the TCP based on differential DVH data
  '''

  #Define voxel parameters
  density = 3*10**7 #Cell density
  vox_vol = 0.3*0.1953125*0.1953125
  N0 = density*vox_vol
  ctv_size = 285.998

  #Initite the TCP
  TCP = 1

  #Iterate for each voxel data point and multiply the voxel TCP by the overall TCP
  for k in range(len(dos)):
    for l in range(vol[k]):
      TCP *= (np.exp(-N0*SF(dos[k],alph,ab)))**(vox_vol/ctv_size)

  #Return the TCP for the whole tumour
  return TCP

In [ ]:
TCP(diffDos,diffVol,104.330,0.24,10.6)

0.9954034892354034